<a href="https://colab.research.google.com/github/susannelobo/Alexis/blob/main/AI-basedThreatAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install numpy==1.26.4 pandas==2.2.3 matplotlib==3.9.2 plotly==5.24.1 scikit-learn==1.5.2
!pip install streamlit==1.25.0 pyngrok==7.0.0 pdfplumber==0.11.4 cryptography==43.0.1 python-dotenv==1.0.1 sentence-transformers==3.1.1 transformers==4.45.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 86.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 114.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 68.1 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 12.7 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 MB 63.4 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 96.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 99.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 127.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.7 MB/s  0:00:00
  DEPRECATION: Building 'pyngrok' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by

In [ ]:
%%writefile threat_analyzer.py
import pdfplumber
import hashlib
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import torch
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import time, json

# Load NLP Models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment = pipeline("sentiment-analysis")
ner = pipeline("ner", grouped_entities=True)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# -------- PDF EXTRACT --------
def extract_text_from_pdf(file):
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()

# -------- SUMMARIZATION --------
def summarize_text(text, max_len=200):
    chunks = [text[i:i+1024] for i in range(0, len(text), 1024)]
    summaries = [summarizer(c, max_length=max_len, min_length=50, do_sample=False)[0]['summary_text'] for c in chunks]
    return " ".join(summaries)

# -------- SENTIMENT --------
def sentiment_score(text):
    result = sentiment(text[:512])[0]
    return result["label"], result["score"]

# -------- NER --------
def extract_entities(text):
    entities = ner(text[:1000])
    df = pd.DataFrame(entities)
    return df

# -------- CLUSTERING --------
def cluster_reports(texts, n_clusters=3):
    embeddings = embedder.encode(texts)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(embeddings)
    return kmeans.labels_

# -------- BLOCKCHAIN LOGGING --------
def generate_block(data):
    data_json = json.dumps(data, sort_keys=True).encode()
    block_hash = hashlib.sha256(data_json).hexdigest()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    return {"timestamp": timestamp, "hash": block_hash, "data": data}

def verify_blockchain(blockchain):
    for i in range(1, len(blockchain)):
        prev_hash = blockchain[i-1]["hash"]
        current_data = blockchain[i]["data"]
        expected_hash = hashlib.sha256(json.dumps(current_data, sort_keys=True).encode()).hexdigest()
        if blockchain[i]["hash"] != expected_hash:
            return False
    return True

Writing threat_analyzer.py


In [ ]:
%%writefile app.py
import streamlit as st
import os
from threat_analyzer import extract_text_from_pdf, summarize_text, sentiment_score, extract_entities, cluster_reports, generate_block, verify_blockchain
import pandas as pd
import plotly.express as px

# --- App Config ---
st.set_page_config(page_title="Threat Intelligence Analyzer", layout="wide")

# --- Password Gate ---
PASSWORD = "quantumshield"  # default password for Colab
entered_pw = st.text_input("🔐 Enter Access Password:", type="password")
if entered_pw != PASSWORD:
    st.warning("Enter correct password to access.")
    st.stop()

st.title("🧠 Threat Intelligence Report Analyzer")
st.caption("Hybrid NLP + ML + Blockchain-based Threat Report Analysis")

uploaded_files = st.file_uploader("📄 Upload Threat Reports (PDFs)", type=["pdf"], accept_multiple_files=True)

if uploaded_files:
    summaries, sentiments, entities_list, blockchain = [], [], [], []

    for file in uploaded_files:
        with st.spinner(f"Processing {file.name}..."):
            text = extract_text_from_pdf(file)
            summary = summarize_text(text)
            sentiment_label, sentiment_val = sentiment_score(text)
            entities = extract_entities(text)
            block = generate_block({"filename": file.name, "summary": summary, "sentiment": sentiment_label})

            summaries.append(summary)
            sentiments.append((sentiment_label, sentiment_val))
            entities_list.append(entities)
            blockchain.append(block)

    # --- Dashboard Summary ---
    st.subheader("📊 Dashboard Summary")
    df_summary = pd.DataFrame({
        "Filename": [f.name for f in uploaded_files],
        "Sentiment": [s[0] for s in sentiments],
        "Confidence": [s[1] for s in sentiments],
        "Summary": summaries
    })
    st.dataframe(df_summary, use_container_width=True)

    # Sentiment Plot
    fig = px.bar(df_summary, x="Filename", y="Confidence", color="Sentiment", title="Sentiment Confidence per Report")
    st.plotly_chart(fig, use_container_width=True)

    # Cluster Reports
    st.subheader("🧩 Clustering Reports")
    labels = cluster_reports(summaries)
    df_summary["Cluster"] = labels
    st.dataframe(df_summary[["Filename", "Cluster", "Sentiment"]])

    # Blockchain Integrity
    st.subheader("🔗 Blockchain Log")
    st.json(blockchain)
    if verify_blockchain(blockchain):
        st.success("✅ Blockchain integrity verified — data is secure.")
    else:
        st.error("❌ Blockchain integrity compromised.")

Writing app.py


In [ ]:
!ngrok authtoken 341YcNuihSmtvKvv42xXjYHaUJI_6TwMASZyweYG5TSdqipcY

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
import threading, os, time

def run_app():
    os.system("streamlit run app.py --server.port 8501 --server.enableCORS=false")

thread = threading.Thread(target=run_app)
thread.start()
time.sleep(5)

public_url = ngrok.connect(8501, bind_tls=True)
print("🚀 Your Streamlit app is live at:", public_url)

🚀 Your Streamlit app is live at: NgrokTunnel: "https://d94cdcc67540.ngrok-free.app" -> "http://localhost:8501"
